In [ ]:
import random
import itertools

In [ ]:
def logic(inputs):
    out1 = inputs[0] or inputs[1]  #or
    out2 = not((not(out1) and inputs[2]) or (out1 and not(inputs[2])))  #xnor
    out3 = not(out2 and inputs[3])   #nand
    out4 = (not(out3) and inputs[4]) or (out3 and not(inputs[4])) #xor
    out5 = not(out4 and inputs[5]) #nand
    out6 = not(out5 and inputs[6])   #nand
    out7 = not(out6 or inputs[7])  #nor
    out8 = out7 and inputs[8] #and
    out9 = not(out8 or inputs[9]) #nor
    out10 = not((not(out9) and inputs[10]) or (out9 and not(inputs[10])))  #xnor

    return out10

Truth table

In [ ]:
# fullRules = [[[False, False, False, False], False],
#              [[False, False, False, True], False],
#              [[False, False, True, False], False],
#              [[False, False, True, True], True],
#              [[True, False, False, False], False],
#              [[True, False, False, True], True],
#              [[True, False, True, False], False],
#              [[True, False, True, True], True]]

fullRules = []
outputTable = []
inputTable = list(itertools.product([False, True], repeat=11))

# for i in range(0, 1023):
#    outputTable.append(random.choice([True, False]))
#   outputTable.append(False)
# outputTable.append(True)

for i in range(0, 2048):
  fullRules.append([inputTable[i], logic(inputTable[i])])
print(fullRules)

[[(False, False, False, False, False, False, False, False, False, False, False), False], [(False, False, False, False, False, False, False, False, False, False, True), True], [(False, False, False, False, False, False, False, False, False, True, False), True], [(False, False, False, False, False, False, False, False, False, True, True), False], [(False, False, False, False, False, False, False, False, True, False, False), False], [(False, False, False, False, False, False, False, False, True, False, True), True], [(False, False, False, False, False, False, False, False, True, True, False), True], [(False, False, False, False, False, False, False, False, True, True, True), False], [(False, False, False, False, False, False, False, True, False, False, False), False], [(False, False, False, False, False, False, False, True, False, False, True), True], [(False, False, False, False, False, False, False, True, False, True, False), True], [(False, False, False, False, False, False, False, Tru

In [ ]:
import csv

f = open('truth_table.csv', 'w+')

with f:
    writer = csv.writer(f)
    for inp in inputTable:
        tmp = []
        for item in inp:
            tmp.append(item)
        tmp.append(logic(inp))
        writer.writerow(tmp)

In [ ]:
P_crossover = 0.7
P_mutation = 0.1
CROSSOVERPOINTSNUMBER = 1
num_of_gates = 10
FIRSTPOPULATION = 1000
SELECTIONPOPULATION = 500
RANDOMPOPULATION = 500

## Creating Basic Gates

Now we define our source test different inputs.


In [ ]:
class Source:
  def __init__(self, value):
    self.value = value
  def get_output(self):
    return self.value


First of all we need to define some basic gates and source classes to create the gates.

In [ ]:
from abc import ABC

class Node(ABC):
    def __init__(self, inputA, inputB):
        self.inputA = inputA
        self.inputB = inputB

    def get_output(self):
        pass

Then we create the "OR" , "AND" and "XOR" gates which implement the GateWith2Inputs class. Then we create the "NOT" gate which only has one input therefore we ignore other input.

In [ ]:
class And(Node):
    def __init__(self, inputA, inputB):
        super().__init__(inputA, inputB)
    def get_output(self):
        return (self.inputA and self.inputB)


class Or(Node):
    def __init__(self, inputA, inputB):
        super().__init__(inputA, inputB)
    def get_output(self):
        return (self.inputA or self.inputB)


class Xor(Node):
    def __init__(self, inputA, inputB):
        super().__init__(inputA, inputB)
    def get_output(self):
        return (not(self.inputA) and self.inputB) or (self.inputA and not(self.inputB))


class Xnor(Node):
    def __init__(self, inputA, inputB):
        super().__init__(inputA, inputB)
    def get_output(self):
        return not((not self.inputA and self.inputB) or (self.inputA and not(self.inputB)))

class Nand(Node):
    def __init__(self, inputA, inputB):
        super().__init__(inputA, inputB)
    def get_output(self):
        return not(self.inputA and self.inputB)


class Nor(Node):
    def __init__(self, inputA, inputB):
        super().__init__(inputA, inputB)
    def get_output(self):
        return not(self.inputA or self.inputB)


In [ ]:
print(Xor(False, False).get_output(),Xor(True, False).get_output(),Xor(False, True).get_output(),Xor(True, True).get_output())

False True True False


In [ ]:
gates = [Or, And, Xor, Xnor, Nand, Nor]

## Circuit Structure


We keep fitness and output of circuit in circuit class. Then we operate population, crossover and mutation.

In [ ]:
class circuit:
  def __init__(self, gateTypes = []):
    self.gateTypes = gateTypes
    self.fitness = None
  def get_fitness(self):
    if (self.fitness == None):
      fit_counter = 0  
      for rule in fullRules:
        sources = rule[0]
        table_output = rule[1]
        circuit_output = self.get_output(sources)
        if (table_output == circuit_output):
          fit_counter += 1
      self.fitness = fit_counter
    return self.fitness
  def get_output(self, sources):
    node = self.gateTypes[0](Source(sources[0]).get_output(), Source(sources[1]).get_output())
    for i in range(0, (len(self.gateTypes) - 1)):
    #   print("i", i, "len source", len(sources), "len gates", len(self.gateTypes))
      tmp = self.gateTypes[i + 1](node.get_output(), sources[i + 2])
      node = tmp
    return node.get_output()
  def get_type_inputs(self):
    return self.gateTypes


In [ ]:
def get_population(num):
  circuits = []
  chroms = []
  while len(chroms) < num:
    temp = [random.choice(gates) for i in range(num_of_gates)]  
    if chroms.count(temp) == 0:
      chroms.append(temp)
      circuits.append(circuit(temp))
  return circuits

In [ ]:
def crossOver(parent1, parent2):
    crossOverPoints = random.sample(range(1, len(parent1)-1), CROSSOVERPOINTSNUMBER)
    crossOverPoints.append(len(parent1))
    child1 = []
    child2 = []
    previousPoint = 0
    decussate = 0
    for point in crossOverPoints:
        parent1Part = parent1[previousPoint:point]
        parent2Part = parent2[previousPoint:point]
        if (decussate % 2) == 0:
            for part in parent1Part:
                child1.append(part)
            for part in parent2Part:
                child2.append(part)
        else:
            for part in parent1Part:
                child1.append(part)
            for part in parent2Part:
                child2.append(part)
        previousPoint = point
        decussate += 1
    # print('child1', child1)
    # print('child2', child2)
    circuit1 = circuit(child1)
    circuit2 = circuit(child2)
    return circuit1, circuit2

In [ ]:
PROBABILITYOFEACHGATE = 0.7
def mutation(child):
    for i in range(len(child)):
        probability = random.uniform(0,1)
        if probability >= PROBABILITYOFEACHGATE:
            randomGateIndex = random.randrange(len(gates))
            child[i] = gates[randomGateIndex]
    Circuit = circuit(child)
    return Circuit


In [ ]:
def next_gen(mating_pool):

    new_gen = []
    for i in range(RANDOMPOPULATION):
        mating_pool.append(circuit([random.choice(gates) for i in range(num_of_gates)]) )
    

    for i in range(0, len(mating_pool), 2):
        crossoverProbability = random.uniform(0,1)
        mutationProbability = random.uniform(0,1)

        if crossoverProbability <= P_crossover:
            child1, child2 = crossOver(mating_pool[i].get_type_inputs(), mating_pool[i+1].get_type_inputs())
            new_gen.append(child1)
            new_gen.append(child2)
        else:
            new_gen.append(mating_pool[i])
            new_gen.append(mating_pool[i+1])

    for i in range(len(new_gen)):
        if (mutationProbability <= P_mutation):
            new_gen[i] = mutation(new_gen[i].get_type_inputs())

    return new_gen

In [ ]:
def calculateBestFitnessOfGeneration(generation):
    bestFitness = 0
    tempFitness = 0
    bestIndividual = circuit([])
    for individual in generation:
        tempFitness = individual.get_fitness()
        if tempFitness > bestFitness:
            bestFitness = tempFitness
            bestIndividual = individual
    print("****", bestIndividual.get_type_inputs())
    print("**** fitness: ", bestIndividual.get_fitness())
    return bestFitness

In [ ]:
def selection(population):
    sortedByFitness = sorted(population, key = lambda x : x.get_fitness(), reverse = True)
    rankSum = (len(sortedByFitness)*(len(sortedByFitness)+1))/2
    ranks = []
    matingPool = []
    for i in range(len(sortedByFitness)):
        rankP = 1/ (((i+1) / rankSum) * 100)
        ranks.append(rankP)
        # print("fitness:", sortedByFitness[i].get_fitness())
        # print("probability", rankP)
    matingPool = random.choices(sortedByFitness, weights = ranks, k = SELECTIONPOPULATION)
    
    return matingPool
    

In [ ]:
def main():
    population = get_population(FIRSTPOPULATION)
    # sortedByFitness = sorted(population, key = lambda x : x.get_fitness(), reverse = True)
    # rankSum = (len(sortedByFitness)*(len(sortedByFitness)+1))/2
    # ranks = []
    # matingPool = []
    # for i in range(len(sortedByFitness)):
    #     rankP = 1/ (round( ((i+1) / rankSum) * 100, 3))
    #     ranks.append(rankP)
    #     print("fitness:", sortedByFitness[i].get_fitness())
    #     print("probability", rankP)
    # matingPool = random.choices(sortedByFitness, weights = ranks, k = 90)
    # for a in sortedByFitness:
    #     print(a.get_fitness())
    # print("------------------")
    # for b in matingPool:
    #     print(b.get_fitness())
    matingPool = selection(population)
    print(matingPool)
    ffitness = 0 
    # new_gen = next_gen(matingPool)
    counter = 1
    while (ffitness < 2048):
        print("generation numebr: ", counter)
        new_gen = next_gen(matingPool)
        ffitness = calculateBestFitnessOfGeneration(new_gen)
        print("fitness is = ", ffitness)
        matingPool = selection(new_gen)
        counter += 1
        
        
main()

[<__main__.circuit object at 0x7f8dd02a26d8>, <__main__.circuit object at 0x7f8dd02e5240>, <__main__.circuit object at 0x7f8dd02f54e0>, <__main__.circuit object at 0x7f8dd1b5b518>, <__main__.circuit object at 0x7f8dd02fb2e8>, <__main__.circuit object at 0x7f8dd02e5ac8>, <__main__.circuit object at 0x7f8dd02f51d0>, <__main__.circuit object at 0x7f8dd02e5ac8>, <__main__.circuit object at 0x7f8dd02ef278>, <__main__.circuit object at 0x7f8dd02e5b70>, <__main__.circuit object at 0x7f8dd02f5780>, <__main__.circuit object at 0x7f8dd02e5390>, <__main__.circuit object at 0x7f8dd02a2048>, <__main__.circuit object at 0x7f8dd02a2518>, <__main__.circuit object at 0x7f8dd02e5ac8>, <__main__.circuit object at 0x7f8dd030e390>, <__main__.circuit object at 0x7f8dd02e5ac8>, <__main__.circuit object at 0x7f8dd1b3bc88>, <__main__.circuit object at 0x7f8dd02e5ac8>, <__main__.circuit object at 0x7f8dd02a2518>, <__main__.circuit object at 0x7f8dd02fb390>, <__main__.circuit object at 0x7f8dd02a71d0>, <__main__

In [ ]:
print(Xor(False, False).get_output(),Xor(True, False).get_output(),Xor(False, True).get_output(),Xor(True, True).get_output())

False True True False


In [ ]:
print(Xnor(False, False).get_output(),Xnor(True, False).get_output(),Xnor(False, True).get_output(),Xnor(True, True).get_output())

True False False True


In [ ]:
~False

-1

In [ ]:
~True

-2

In [ ]:
not False

True